In [8]:
import pandas as pd
import numpy as np
import requests as re
import json as js
import time
from bs4 import BeautifulSoup as bs
import asyncio

In [2]:
with open("authorization.json") as f:
    key = js.load(f)
    NOTION_TOKEN = key["NOTION_TOKEN"]
    DATABASE_ID = key["DATABASE_ID"]
    headers = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"}

In [11]:
def create_page(data: dict):
    create_url = "https://api.notion.com/v1/pages"
    payload = {"parent": {"database_id": DATABASE_ID}, "properties": data}
    res = re.post(create_url, headers=headers, json=payload)
    return res

user_key = "squad7"
username = "squad7"
pwd = "squad7"

create_page({"key": {"title": [{"text": {"content": user_key}}]}, "name": {"rich_text": [{"text": {"content": username}}]}, "password": {"rich_text": [{"text": {"content": pwd}}]}})

In [78]:
def get_pages(filter_username = None, num_pages=None):
    url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"

    get_all = num_pages is None
    page_size = 100 if get_all else num_pages

    if filter_username:
        payload = {"page_size": page_size, 'filter': {'property': 'name', 'rich_text': {'contains': filter_username}}}
    else:
        payload = {"page_size": page_size}
    response = re.post(url, json=payload, headers=headers)

    data = response.json()

    results = data["results"]
    while data["has_more"] and get_all:
        payload = {"page_size": page_size, "start_cursor": data["next_cursor"]}
        url = f"https://api.notion.com/v1/databases/{DATABASE_ID}/query"
        response = re.post(url, json=payload, headers=headers)
        data = response.json()
        results.extend(data["results"])

    return results

In [91]:
key = []
names = []
password = []
for page in get_pages():
    key.append(page['properties']['key']['title'][0]['text']['content'])
    names.append(page['properties']['name']['rich_text'][0]['text']['content'])
    password.append(page['properties']['password']['rich_text'][0]['text']['content'])